# Separator Calculations

In [30]:
import pandas as pd
import numpy as np
from pvtpy.compositional import Chromatography, Component, properties_df
from pvtpy.units import Pressure, Temperature

In [31]:
properties_df.index

Index(['methane', 'ethane', 'propane', 'isobutane', 'butane', 'isopentane',
       'pentane', 'neopentane', 'n-hexane', '2-methyl-pentane',
       '3-methyl-penthane', 'neohexane', '2-3-dimethyl-butane', 'n-heptane',
       '2-methyl-hexane', '3-methyl-hexane', '3-ethylpentane',
       '2-2-dimethyl-pentane', '2-4-dimethyl-pentane', '3-3-dimethyl-pentane',
       'triptane', 'n-octane', 'isooctane', 'n-nonane', 'n-decane',
       'cyclopentane', 'methylcyclopentane', 'cyclohexane',
       'methylcyclohexane', 'ethylene', 'propylene', 'cis-2-butene',
       'trans-2-butene', 'isobutene', '1-pentene', '1-2-butadiene',
       '1-3-butadiene', 'isoprene', 'benzene', 'toluene', 'ethylbenzene',
       'o-xylene', 'm-xylene', 'p-xylene', 'styrene', 'isopropylbenzene',
       'methyl-alcohol', 'ethyl-alcohol', 'carbon-monixide', 'carbon-dioxide',
       'hydrogen-sulfide', 'sulfur-dioxide', 'ammonia', 'hydrogen', 'oxygen',
       'nitrogen', 'chlorine', 'water', 'n-undecane', 'n-dodecane',
   

In [32]:
d1 = {
    'comp': ['carbon-dioxide','nitrogen','methane','ethane','propane','isobutane','butane','isopentane','pentane','n-hexane'],
    'mole_fraction':[0.0008,0.0164,0.2840,0.0716,0.1048,0.042,0.042,0.0191,0.01912,0.0405]
}

c7_plus = Component(
    name = 'C7+', 
    molecular_weight=252,
    specific_gravity = 0.8429,
    mole_fraction=0.3597,   
    critical_pressure=140,
    critical_pressure_unit='psi',
    critical_temperature=1279.8,
    critical_temperature_unit='rankine',
    params = {'acentric_factor':0.5067}
)

ch1 = Chromatography()
ch1.from_df(pd.DataFrame(d1),name='comp')
ch1.plus_fraction = c7_plus

ch1.df()['mole_fraction']

carbon-dioxide    0.000800
nitrogen          0.016400
methane           0.283994
ethane            0.071599
propane           0.104798
isobutane         0.041999
butane            0.041999
isopentane        0.019100
pentane           0.019120
n-hexane          0.040499
C7+               0.359693
Name: mole_fraction, dtype: float64

In [33]:
ma = ch1.apparent_molecular_weight()
print(f'Aparent Molecular weight {ma}')

Aparent Molecular weight 113.61639653206936


In [34]:
rho = 44.794

## Stage 1

In [35]:
p1 = Pressure(value=400, unit='psi')
t1 = Temperature(value=72, unit='farenheit')

ch1.equilibrium_ratios(p1,t1,method='whitson')

carbon-dioxide     2.794898
nitrogen          30.262636
methane           12.523440
ethane             1.911981
propane            0.489915
isobutane          0.197277
butane             0.139279
isopentane         0.057828
pentane            0.044560
n-hexane           0.015517
C7+                0.000017
Name: k, dtype: float64

In [36]:
fsh1, phase1 = ch1.flash_calculations(p1,t1)
fsh1.index.name = 'component'
print(fsh1)
print(fsh1[['xi','yi']].sum())

                mole_fraction        xi        yi          k
component                                                   
carbon-dioxide       0.000800  0.000592  0.001301   2.198146
nitrogen             0.016400  0.001713  0.051773  30.229625
methane              0.283994  0.069850  0.799758  11.449594
ethane               0.071599  0.063289  0.091612   1.447522
propane              0.104798  0.130747  0.042300   0.323526
isobutane            0.041999  0.056641  0.006735   0.118908
butane               0.041999  0.057502  0.004662   0.081068
isopentane           0.019100  0.026688  0.000822   0.030816
pentane              0.019120  0.026801  0.000620   0.023133
n-hexane             0.040499  0.057142  0.000414   0.007246
C7+                  0.359693  0.509035  0.000002   0.000004
xi    1.0
yi    1.0
dtype: float64


In [37]:
moles_stage1 = ch1.phase_moles(p1,t1)
moles_stage1

{'liquid_moles': 0.706615190417923, 'gas_moles': 0.293384809582077}

## Stage 2

In [38]:
p2 = Pressure(value=350, unit='psi')
t2 = Temperature(value=72, unit='farenheit')

In [39]:
ch2 = Chromatography()
ch2.from_df(fsh1, mole_fraction='xi')

c7_plus1 = Component(
    name = 'C7+', 
    molecular_weight=252,
    specific_gravity = 0.8429,
    mole_fraction=fsh1.loc['C7+','xi'],   
    critical_pressure=140,
    critical_pressure_unit='psi',
    critical_temperature=1279.8,
    critical_temperature_unit='rankine',
    params = {'acentric_factor':0.5067}
)
ch2.plus_fraction = c7_plus1


ch2.df()['mole_fraction']

/Users/scuervo91/Documents/dev/pvtpy/pvtpy/compositional/chromatography.py:48: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  self.components = parse_obj_as(List[Component], _merged.to_dict(orient='records'))


carbon-dioxide    0.000592
nitrogen          0.001713
methane           0.069850
ethane            0.063289
propane           0.130747
isobutane         0.056641
butane            0.057502
isopentane        0.026688
pentane           0.026801
n-hexane          0.057142
C7+               0.509035
Name: mole_fraction, dtype: float64

In [40]:
moles_stage2 = ch2.phase_moles(p2,t2)
moles_stage2

{'liquid_moles': 0.9869509099020275, 'gas_moles': 0.013049090097972503}

In [41]:
fsh2, phase2 = ch2.flash_calculations(p2,t2)
fsh2.index.name = 'component'
print(fsh2)
print(fsh2[['xi','yi']].sum())

                mole_fraction        xi        yi          k
component                                                   
carbon-dioxide       0.000592  0.000580  0.001458   2.512167
nitrogen             0.001713  0.001191  0.041154  34.548143
methane              0.069850  0.060335  0.789504  13.085251
ethane               0.063289  0.062753  0.103813   1.654311
propane              0.130747  0.131831  0.048744   0.369743
isobutane            0.056641  0.057287  0.007785   0.135895
butane               0.057502  0.058191  0.005391   0.092649
isopentane           0.026688  0.027029  0.000952   0.035219
pentane              0.026801  0.027145  0.000718   0.026438
n-hexane             0.057142  0.057892  0.000479   0.008281
C7+                  0.509035  0.515766  0.000002   0.000005
xi    1.0
yi    1.0
dtype: float64


## Stage 3

In [42]:
p3 = Pressure(value=14.7, unit='psi')
t3 = Temperature(value=60, unit='farenheit')

In [43]:
ch3 = Chromatography()
ch3.from_df(fsh2.reset_index(),name = fsh2.index.name, mole_fraction='xi')

c7_plus3 = Component(
    name = 'C7+', 
    molecular_weight=252,
    specific_gravity = 0.8429,
    mole_fraction=fsh2.loc['C7+','xi'],   
    critical_pressure=140,
    critical_pressure_unit='psi',
    critical_temperature=1279.8,
    critical_temperature_unit='rankine',
    params = {'acentric_factor':0.5067}
)
ch3.plus_fraction = c7_plus3


ch3.df()['mole_fraction']

/Users/scuervo91/Documents/dev/pvtpy/pvtpy/compositional/chromatography.py:48: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  self.components = parse_obj_as(List[Component], _merged.to_dict(orient='records'))


carbon-dioxide    0.000580
nitrogen          0.001191
methane           0.060335
ethane            0.062753
propane           0.131831
isobutane         0.057287
butane            0.058191
isopentane        0.027029
pentane           0.027145
n-hexane          0.057892
C7+               0.515766
Name: mole_fraction, dtype: float64

In [44]:
moles_stage3 = ch3.phase_moles(p3,t3)
moles_stage3

{'liquid_moles': 0.7146137415095994, 'gas_moles': 0.2853862584904005}

In [45]:
fsh3, phase3 = ch3.flash_calculations(p3,t3)
fsh3.index.name = 'component'
print(fsh3)
print(fsh3[['xi','yi']].sum())

                mole_fraction        xi        yi           k
component                                                    
carbon-dioxide       0.000580  0.000038  0.001939   51.134100
nitrogen             0.001191  0.000005  0.004161  778.464695
methane              0.060335  0.000729  0.209590  287.336282
ethane               0.062753  0.005989  0.204892   34.210574
propane              0.131831  0.046825  0.344688    7.361205
isobutane            0.057287  0.038984  0.103116    2.645045
butane               0.058191  0.047590  0.084736    1.780549
isopentane           0.027029  0.029919  0.019790    0.661425
pentane              0.027145  0.031750  0.015615    0.491810
n-hexane             0.057892  0.076449  0.011424    0.149439
C7+                  0.515766  0.721721  0.000050    0.000069
xi    1.0
yi    1.0
dtype: float64


In [46]:
moles_stages = [moles_stage1,moles_stage2,moles_stage3]

nl = 1
for i in moles_stages:
    nl *= i['liquid_moles']
    
nv = 1 - nl 

print(f'liquid Moles Stock Tank {nl}\nLiberated Gas Moles {nv}')

liquid Moles Stock Tank 0.49836769662173136
Liberated Gas Moles 0.5016323033782686


In [47]:
ch4 = Chromatography()
ch4.from_df(fsh3.reset_index(),name = fsh3.index.name, mole_fraction='xi')

c7_plus4 = Component(
    name = 'C7+', 
    molecular_weight=252,
    specific_gravity = 0.8429,
    mole_fraction=fsh3.loc['C7+','xi'],   
    critical_pressure=140,
    critical_pressure_unit='psi',
    critical_temperature=1279.8,
    critical_temperature_unit='rankine',
    params = {'acentric_factor':0.5067}
)
ch4.plus_fraction = c7_plus4


ch4.df()['mole_fraction']

/Users/scuervo91/Documents/dev/pvtpy/pvtpy/compositional/chromatography.py:48: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  self.components = parse_obj_as(List[Component], _merged.to_dict(orient='records'))


carbon-dioxide    0.000038
nitrogen          0.000005
methane           0.000729
ethane            0.005989
propane           0.046825
isobutane         0.038984
butane            0.047590
isopentane        0.029919
pentane           0.031750
n-hexane          0.076449
C7+               0.721721
Name: mole_fraction, dtype: float64

In [48]:
ch4.apparent_molecular_weight()

200.23335136754622

In [49]:
## Separator Functions

In [50]:
from pvtpy.compositional import Stage, SeparatorTest

In [51]:

stage1 = Stage(
    pressure=p1,
    temperature = t1
)

stage2 = Stage(
    pressure=p2,
    temperature = t2
)

stage3 = Stage(
    pressure=p3,
    temperature = t3
)

list_stages = [stage1, stage2, stage3]

sep = SeparatorTest(
    initial_chromatography = ch1,
    stages = list_stages
)


In [52]:
sep.solve()

Stage 0
Stage 1
Stage 2


True

In [53]:
sep.stages[-1].phase_moles

PhaseMoles(liquid_moles=0.7146137415095994, gas_moles=0.2853862584904005)

Calculate apparent molecular weight of the stock-tank oil from its composition, to
give

In [54]:
sep.stages[-1].chromatography.apparent_molecular_weight()

200.23335136754622

Calculate the actual number of moles of the liquid phase at the stock-tank condi-
tions

Calculate the total number of moles of the liberated gas


In [55]:
sep.final_moles()

PhaseMoles(liquid_moles=0.49836769662173136, gas_moles=0.5016323033782686)

In [56]:
sep.final_molecular_weight()

200.23335136754622

In [57]:
rho = 50.920
sep.gas_solubility(rho=50.920)

545.2989698764427

In [58]:
sep.volumetric_factor(44.794,50.920)

1.2942651571380284